In [2]:
!pip install contextualized-topic-models==2.2.0
!pip install pyldavis
!pip install konlpy

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.



  Using cached contextualized_topic_models-2.2.0-py2.py3-none-any.whl (33 kB)
  Using cached ipywidgets-7.5.1-py2.py3-none-any.whl (121 kB)
  Using cached ipython-7.16.1-py3-none-any.whl (785 kB)
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Using cached ipykernel-6.15.1-py3-none-any.whl (132 kB)
  Using cached ipykernel-6.15.0-py3-none-any.whl (133 kB)
  Using cached ipykernel-6.14.0-py3-none-any.whl (133 kB)
  Using cached ipykernel-6.13.1-py3-none-any.whl (133 kB)
  Using cached ipykernel-6.13.0-py3-none-any.whl (131 kB)
  Using cached ipykernel-6.12.1-py3-none-any.whl (130 kB)
  Using cached ipykernel-6.12.0-py3-none-any.whl (130 kB)
  Using cached ipykernel-6.11.0-py3-none-any.whl (130 kB)
  Using cached ipykernel-6.10.0-py3-none-any.whl (130 kB)
  Using cached ipykernel-6.9.2-py3-none-any.whl (130 kB)
  Using cached ipykernel-6.9.0-py3-none-any.whl (128 kB)
  Using cached ipykernel-6.8.0-py3-none-any.whl (128 kB)
  Using cached ipykernel-6.7.0-py3-none-any.whl (127

  Using cached pyLDAvis-3.3.1.tar.gz (1.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Using cached funcy-1.17-py2.py3-none-any.whl (33 kB)
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136882 sha256=8490268f1a59cae9f09268b6d59d7e7d06c6d22f437f54ee04264edd5cecfb12
  Stored in directory: c:\users\notebiz003\appdata\local\pip\cache\wheels\57\a4\86\d10c6c2e0bf149fbc0afb0aa5a6528ac35b30a133a0270c477
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=fbf25b97b7fc58df4477727a25ad7de504a45855078ddaea8ae71eb28

In [3]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Kkma
from tqdm import tqdm
import pandas as pd

In [4]:
DATA_PATH = 'kpfSBERT_Clustering\data/newstrust_20210601_samlple.json'
df = pd.read_json(DATA_PATH, encoding='utf-8')
documents = list(df['title'])

In [5]:
preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

100%|███████████████████████████████████████████████████████████████████████| 12006/12006 [00:00<00:00, 1146087.98it/s]


In [6]:
len(preprocessed_documents)

12006

In [7]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [8]:
custom_tokenizer = CustomTokenizer(Kkma())

In [9]:
import platform
print(platform.architecture())

('64bit', 'WindowsPE')


In [10]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [11]:
train_bow_embeddings = vectorizer.fit_transform(preprocessed_documents)

In [12]:
print(train_bow_embeddings.shape)

(12006, 3000)


In [13]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

C:\Users\Notebiz003\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
train_contextualized_embeddings = bert_embeddings_from_list(preprocessed_documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/61 [00:00<?, ?it/s]

In [ ]:
qt = TopicModelDataPreparation()
training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

In [ ]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=50, num_epochs=20)
ctm.fit(training_dataset)

In [ ]:
ctm.get_topics(5)

In [ ]:
ctm.get_topics(10)

In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)